In [16]:
# from src.crawler import Crawler

In [17]:
CRWALING = True         # 크롤링
DB_BULK_INSERT = True   # 크롤링 데이터 DB INSERT

TEST_SIZE = 1           # 크롤링 공고 테스트용 사이트 설정되어 있는 만큼 진행됨 기본값 -1 = 전체 진행
batch_id = ''           # 중간 진입위한 BATCH_ID 넣으면 

if batch_id:
    CRWALING = False
    DB_BULK_INSERT = False

In [18]:
# ============================================
# 환경 변수 로드 및 필수 패키지 자동 설치
# ============================================
import os
import sys
import subprocess
from pathlib import Path

# 필수 패키지 목록 (버전 지정 없이 설치)
REQUIRED_PACKAGES = [
    'python-dotenv',
    'beautifulsoup4',  # bs4
    'requests',
    'psycopg2-binary',
    'pandas',
    'numpy',
    'llama-cloud-services',  # LlamaParse 사용
    'langchain-openai',  # 임베딩 사용
]

# 패키지 설치 함수 (오류 발생해도 계속 진행)
def install_package(package_name):
    """패키지가 없으면 설치 (오류 발생해도 계속 진행)"""
    try:
        # 패키지 사용 가능 여부 확인
        if package_name == 'beautifulsoup4':
            import bs4
            print(f"✅ {package_name} (bs4) 사용 가능")
            return True
        elif package_name == 'python-dotenv':
            import dotenv
            print(f"✅ {package_name} 사용 가능")
            return True
        elif package_name == 'psycopg2-binary':
            import psycopg2
            print(f"✅ {package_name} (psycopg2) 사용 가능")
            return True
        elif package_name == 'llama-cloud-services':
            import llama_cloud_services
            print(f"✅ {package_name} 사용 가능")
            return True
        elif package_name == 'langchain-openai':
            import langchain_openai
            print(f"✅ {package_name} 사용 가능")
            return True
        else:
            module_name = package_name.replace('-', '_')
            __import__(module_name)
            print(f"✅ {package_name} 사용 가능")
            return True
    except ImportError:
        # 패키지 설치 시도
        print(f"📦 {package_name} 설치 시도 중...")
        try:
            # 버전 충돌을 피하기 위해 --no-deps 옵션 없이 설치
            # 오류가 발생해도 계속 진행
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", package_name, "--quiet"],
                capture_output=True,
                text=True
            )
            if result.returncode == 0:
                print(f"✅ {package_name} 설치 완료")
                return True
            else:
                print(f"⚠️ {package_name} 설치 실패 (의존성 충돌 가능)")
                print(f"   수동 설치 필요: pip install {package_name}")
                return False
        except Exception as e:
            print(f"⚠️ {package_name} 설치 중 오류: {str(e)[:50]}")
            return False

# 필수 패키지 설치 확인 및 설치
print("=" * 50)
print("필수 패키지 확인 중...")
print("=" * 50)
installed_count = 0
for package in REQUIRED_PACKAGES:
    if install_package(package):
        installed_count += 1
print("=" * 50)
print(f"패키지 확인 완료: {installed_count}/{len(REQUIRED_PACKAGES)}")

# pgvector는 별도로 처리 (의존성 충돌 가능성 높음)
try:
    import pgvector
    print("✅ pgvector 사용 가능")
except ImportError:
    print("⚠️ pgvector 없음 (필요시 수동 설치: pip install pgvector)")

print("=" * 50)

# dotenv 로드
from dotenv import load_dotenv

# .env 파일 위치 확인 (프로젝트 루트)
project_root = Path.cwd().parent if Path.cwd().name == 'zf_crawler' else Path.cwd()
env_file = project_root / '.env'

if env_file.exists():
    print(f"✅ .env 파일 발견: {env_file}")
    load_dotenv(env_file)
else:
    print(f"⚠️ .env 파일 없음: {env_file}")
    print("환경 변수는 시스템 환경 변수에서 로드됩니다.")
    load_dotenv()  # 상위 디렉토리에서도 찾음

print(f"\n현재 작업 디렉토리: {os.getcwd()}")
print(f"프로젝트 루트: {project_root}")
print("=" * 50)

필수 패키지 확인 중...
✅ python-dotenv 사용 가능
✅ beautifulsoup4 (bs4) 사용 가능
✅ requests 사용 가능
✅ psycopg2-binary (psycopg2) 사용 가능
✅ pandas 사용 가능
✅ numpy 사용 가능
✅ llama-cloud-services 사용 가능
✅ langchain-openai 사용 가능
패키지 확인 완료: 8/8
✅ pgvector 사용 가능
✅ .env 파일 발견: /Users/kimjm/Desktop/ZIP-FIT-2/.env

현재 작업 디렉토리: /Users/kimjm/Desktop/ZIP-FIT-2/zf_crawler
프로젝트 루트: /Users/kimjm/Desktop/ZIP-FIT-2


## 1. 테이블 존재 여부 확인

In [ ]:
# # db 테이블 초기화
# RESET_DB = True

# if RESET_DB:
#     from src.database import DataBaseHandler

#     with DataBaseHandler() as db:
#         # 1. 청크 데이터 삭제
#         db.execute_query("TRUNCATE TABLE doc_chunks RESTART IDENTITY CASCADE;", fetch_one=False)
#         print("✅ doc_chunks 초기화 완료")
        
#         # 2. 파일 데이터 삭제
#         db.execute_query("TRUNCATE TABLE annc_files RESTART IDENTITY CASCADE;", fetch_one=False)
#         print("✅ annc_files 초기화 완료")
        
#         # 3. 공고 데이터 삭제
#         db.execute_query("TRUNCATE TABLE annc_all RESTART IDENTITY CASCADE;", fetch_one=False)
#         print("✅ annc_all 초기화 완료")
        
#         # 4. 임시 테이블 삭제
#         db.execute_query("TRUNCATE TABLE annc_lh_temp RESTART IDENTITY CASCADE;", fetch_one=False)
#         print("✅ annc_lh_temp 초기화 완료")

#     print("\n🎉 모든 테이블 초기화 완료! annc_id가 1부터 시작됩니다.")
# else:
#     print("⏭️ DB 초기화 건너뜀")

쿼리 실행 완료. 영향 받은 행 수: -1
✅ doc_chunks 초기화 완료
쿼리 실행 완료. 영향 받은 행 수: -1
✅ annc_files 초기화 완료
쿼리 실행 완료. 영향 받은 행 수: -1
✅ annc_all 초기화 완료
쿼리 실행 완료. 영향 받은 행 수: -1
✅ annc_lh_temp 초기화 완료

🎉 모든 테이블 초기화 완료! annc_id가 1부터 시작됩니다.


In [20]:
from src.database import DataBaseHandler

tables = ("annc_all","annc_files","annc_lh_temp","doc_chunks")

cases = ','.join([f"""sum(case when table_name in ('{table}') then 1 else 0 end) as {table}""" for table in tables])

query = f"""
SELECT 
{cases}
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_name = any(%s)
"""

with DataBaseHandler() as db:
    result = db.execute_query(query, (list(tables),))

zero_count = 0
for val in result[0].items():
    table_name = val[0]
    table_count = val[1]
    print(f"{val[0]}: {'✅' if val[1] > 0 else '❌'}")
    if table_count == 0:
        zero_count += 1
if zero_count > 0:
    print("테이블 이상 상태 확인 필요")
    raise

annc_all: ✅
annc_files: ✅
annc_lh_temp: ✅
doc_chunks: ✅


## 2. 크롤링

In [21]:
# 크롤링 본 로직

from src.crawler.lh import LH


lh_crwaler = LH()

if CRWALING:
    df_all_annc = lh_crwaler.crawl_lh_notices_all_data('접수중')
    df_all_annc += lh_crwaler.crawl_lh_notices_all_data('공고중')
    # df_all_annc += lh_crwaler.crawl_lh_notices_all_data('정정공고중')
else:
    df_all_annc = []
    print("크롤링 x")

-----------------------------------------------------------------------------
🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [유형 '접수중'] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 1000)...
✅ 페이지 1 로드 성공. 13개 중 13개 데이터 추출.
🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.

✨ 전체 크롤링 완료. 총 13건의 데이터를 수집했습니다.
-----------------------------------------------------------------------------
🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [유형 '공고중'] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 1000)...
✅ 페이지 1 로드 성공. 131개 중 131개 데이터 추출.
🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.

✨ 전체 크롤링 완료. 총 131건의 데이터를 수집했습니다.


In [22]:
# 크롤링 된 데이터 중복제거

len_before = len(df_all_annc)
unique_tuples = set(tuple(sorted(d.items())) for d in df_all_annc)
df_all_annc = [dict(t) for t in unique_tuples]
len_after = len(df_all_annc)

print(f'중복제거 {len_before} ➡️ {len_after}')

중복제거 144 ➡️ 131


In [23]:
len_before = len(df_all_annc)
df_all_annc = [annc for annc in df_all_annc if annc['annc_type'] in ('매입','임대')]
len_after = len(df_all_annc)

print(f'기타항목 제거 {len_before} ➡️ {len_after}')

기타항목 제거 131 ➡️ 36


In [24]:
len_before = len(df_all_annc)
df_all_annc = [annc for annc in df_all_annc]
len_after = len(df_all_annc)

print(f'전지역 {len_before} ➡️ {len_after}')

전지역 36 ➡️ 36


In [25]:
df_all_annc

[{'annc_deadline_dt': '2025.12.19',
  'annc_dtl_type': '영구임대',
  'annc_pblsh_dt': '2025.12.04',
  'annc_region': '전북특별자치도',
  'annc_status': '공고중',
  'annc_title': '익산부송1, 익산동산단지 영구임대주택 입주자격완화 예비입주자 모집공고문',
  'annc_type': '임대',
  'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019130&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09',
  'lh_ais_tp_cd': '09',
  'lh_ccr_cnnt_sys_ds_cd': '03',
  'lh_ls_sst': '',
  'lh_pan_id': '2015122300019130',
  'lh_upp_ais_tp_cd': '06'},
 {'annc_deadline_dt': '2025.12.19',
  'annc_dtl_type': '영구임대',
  'annc_pblsh_dt': '2025.11.21',
  'annc_region': '충청북도',
  'annc_status': '공고중',
  'annc_title': '청주산남2-1 영구임대주택 예비입주자 모집 공고(2025.11.21)',
  'annc_type': '임대',
  'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019084&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09',
  'lh_ais_tp_cd': '09',
  'lh_ccr_cnnt_sys_ds_cd': '03',
  'lh_ls_sst': '',
  'lh_pan_id': '

# 3. 대상 DB INSERT - BATCH ID 발행

In [26]:
from src.database.repository import (
    AnncLhRepository, AnncQrRepository,
    AnncAllRepository, AnncFileRepository,
    DocChunkRepository,
)

lh_repo = AnncLhRepository()
# qr_repo = AnncQrRepository()
all_repo = AnncAllRepository()
file_repo = AnncFileRepository()
dc_repo = DocChunkRepository()

if DB_BULK_INSERT: 
    batch_id = lh_repo.bulk_insert_announcements(df_all_annc)
else:
    print("크롤링 데이터 삽입 x")

if not batch_id:
    raise "batch_id가 없으면 진행 불가."


In [27]:
annc_list_lh = AnncQrRepository().get_announcements_merge_target(batch_id, annc_status=['공고중','접수중'])

print(f'{len(annc_list_lh)}건 조회 완료')

30건 조회 완료


# 4. 파일 등록

In [28]:
from datetime import datetime, timedelta
from src.parser import parse_pdf
from src.chunker import create_chunks_from_elements
from src.embedder import embed_chunks
import signal
import json

# 타임아웃 설정 (초)
PDF_PARSE_TIMEOUT = 100  # 100초

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("PDF 파싱 타임아웃")

def parse_pdf_with_timeout(file_path, annc_id, timeout=PDF_PARSE_TIMEOUT):
    """타임아웃이 적용된 PDF 파싱 (signal 사용 - Unix/Mac 전용)"""
    # 기존 핸들러 저장
    old_handler = signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # 타임아웃 설정
    
    try:
        result = parse_pdf(file_path, annc_id)
        signal.alarm(0)  # 타임아웃 해제
        return result
    except TimeoutException:
        raise
    finally:
        signal.alarm(0)  # 타임아웃 해제
        signal.signal(signal.SIGALRM, old_handler)  # 핸들러 복원

def process(row_lh, corp_cd):
    """
    공고 리스트
    """
    time_laps = []

    title_now = lambda title: (title, datetime.now())

    # 1. 배치 테이블 상태 변경 -> 시작
    lh_repo.update_announcements('START', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 시작 - {row_lh['annc_title']}"))

    # 2. 공고 닫기 처리
    row_lh['corp_cd'] = corp_cd
    row_lh['service_status'] = 'CLOSE'
    row_lh['created_at'] = datetime.now()
    row_lh['updated_at'] = datetime.now()

    print(row_lh)
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"공고 닫기 처리"))

    if not merge_result:
        raise Exception("머지된 행 없음")
    
    merge_result = merge_result[0]
    annc_id = merge_result['annc_id']

    # 3. 파일 조회
    file_list = lh_crwaler.get_file_list(row_lh)
    time_laps.append(title_now(f"파일 조회"))

    if not file_list:
        raise Exception("파일 없음")
    
    # 4. 파일, 청크 벡터 초기화
    dc_repo.delete_chunks_by_annc_id(annc_id)
    file_repo.delete_files_by_annc_id(annc_id)
    # print(f"{get_elapsed_time()} {batch_msg} - 파일 초기화")
    time_laps.append(title_now(f"파일 기록 삭제 처리"))

    for idx_file, file_info in enumerate(file_list):
        time_laps.append(title_now(f"파일 처리 시작 ({idx_file+1}/{len(file_list)})"))

        annc_file = {}

        annc_file['annc_id'] = annc_id
        annc_file['file_name'] = file_info['cmnAhflNm']
        annc_file['file_type'] = file_info['slPanAhflDsCdNm']
        annc_file['file_ext'] = 'pdf'

        # 파일 다운
        file_path, annc_file = lh_crwaler.down_file(file_info['cmnAhflSn'], annc_file)
        time_laps.append(title_now(f"파일 다운로드"))

        # 파일 DB 등록
        inserted_file_info = file_repo.bulk_insert_files([annc_file])[0]
        file_id, file_name = inserted_file_info['file_id'], inserted_file_info['file_name']
        time_laps.append(title_now(f"파일 정보 DB 기록"))

        # 파일 엘리먼트 구성 (타임아웃 적용 - signal)
        try:
            parsed = parse_pdf_with_timeout(file_path, annc_id)
        except TimeoutException as e:
            print(f"⏰ PDF 파싱 타임아웃 ({PDF_PARSE_TIMEOUT}초 초과): {annc_file['file_name']}")
            if os.path.exists(file_path):
                os.remove(file_path)
            continue
        except Exception as e:
            print(f"❌ PDF 파싱 오류 ({annc_file['file_name']}): {e}")
            if os.path.exists(file_path):
                os.remove(file_path)
            continue
        time_laps.append(title_now(f"파일 -> Markdown"))
        # test_elements = parsed.elements[:10]

        chunks = create_chunks_from_elements(parsed.elements, annc_id)
        time_laps.append(title_now(f"Markdown -> 청크"))

        embed_chunks(chunks)
        time_laps.append(title_now(f"청크 -> 임베딩"))

        chunk_dto = [{
            'file_id': file_id,
            'annc_id': annc_id,
            'chunk_type': c.element_type,#get('element_type','text'),
            'chunk_text': c.text, #get('text',''),
            'page_num': c.page_number,
            'embedding': c.embedding,
            'metadata': json.dumps(c.metadata)  # dict를 JSON 문자열로 변환
        } for c in chunks]

        
        dc_repo.bulk_insert_chunks(chunk_dto)
        time_laps.append(title_now(f"임베딩 정보 기록"))

        # 사용한 파일 삭제
        if os.path.exists(file_path):
            os.remove(file_path)
        time_laps.append(title_now(f"파일 삭제"))

        # return chunks
    lh_repo.update_announcements('COMPLETE', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 종료"))

    # print(parsed)
    row_lh['service_status'] = 'OPEN'
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"배치 완료 처리"))

    return time_laps

In [29]:
def print_time_laps(time_laps):
    """
    process()의 반환값(time_laps)을 받아
    - 직전 단계 대비 소요시간
    - 시작 이후 총 소요시간
    을 함께 출력한다.
    """
    if not time_laps:
        print("시간 정보가 없습니다.")
        return

    start_time = time_laps[0][1]

    for i, (title, t) in enumerate(time_laps):
        if i == 0:
            diff_prev = timedelta(0)
        else:
            diff_prev = t - time_laps[i-1][1]

        diff_total = t - start_time

        print(f"[{i+1:02d}] {title}")
        print(f"  ├ 전 단계 대비: {diff_prev}")
        print(f"  └ 전체 경과   : {diff_total}")

## 실행 및 타임 랩스

In [30]:
# 공고 단위로 에러 처리하여 문제있는 공고 건너뛰기
success_count = 0
fail_count = 0
failed_anncs = []

for row in annc_list_lh:
    try:
        time_laps = process(row, "LH")
        print_time_laps(time_laps)
        success_count += 1
    except Exception as e:
        fail_count += 1
        failed_anncs.append(row['annc_title'])
        print(f"\n❌ 공고 처리 실패: {row['annc_title']}")
        print(f"   에러: {type(e).__name__}: {str(e)[:150]}")
        print("   → 다음 공고로 넘어갑니다.\n")
        continue

print("\n" + "="*50)
print(f"📊 최종 결과: 성공 {success_count}건, 실패 {fail_count}건")
if failed_anncs:
    print(f"❌ 실패한 공고:")
    for title in failed_anncs:
        print(f"   - {title}")
print("="*50)

{'batch_id': '5636b17d-9d04-4c3b-88a4-150a87d88e51', 'batch_seq': 1, 'annc_title': '익산부송1, 익산동산단지 영구임대주택 입주자격완화 예비입주자 모집공고문', 'annc_url': 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019130&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09', 'batch_status': 'PENDING', 'batch_start_dttm': datetime.datetime(2025, 12, 15, 3, 31, 40, 684992, tzinfo=datetime.timezone.utc), 'annc_type': '임대', 'annc_dtl_type': '영구임대', 'annc_region': '전북특별자치도', 'annc_pblsh_dt': '2025.12.04', 'annc_deadline_dt': '2025.12.19', 'annc_status': '공고중', 'lh_pan_id': '2015122300019130', 'lh_ais_tp_cd': '09', 'lh_upp_ais_tp_cd': '06', 'lh_ccr_cnnt_sys_ds_cd': '03', 'lh_ls_sst': '', 'corp_cd': 'LH', 'service_status': 'CLOSE', 'created_at': datetime.datetime(2025, 12, 15, 12, 31, 40, 999025), 'updated_at': datetime.datetime(2025, 12, 15, 12, 31, 40, 999025)}
[('익산부송1, 익산동산단지 영구임대주택 입주자격완화 예비입주자 모집공고문', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panI

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 1272f006-9179-4c58-90c2-9c75b42d4ef8
[('익산부송1, 익산동산단지 영구임대주택 입주자격완화 예비입주자 모집공고문', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019130&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09', datetime.datetime(2025, 12, 15, 12, 31, 40, 999025), datetime.datetime(2025, 12, 15, 12, 31, 40, 999025), 'LH', '임대', '영구임대', '전북특별자치도', '2025.12.04', '2025.12.19', '공고중', 'OPEN')]
[01] 배치 시작 - 익산부송1, 익산동산단지 영구임대주택 입주자격완화 예비입주자 모집공고문
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.092911
  └ 전체 경과   : 0:00:00.092911
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.116966
  └ 전체 경과   : 0:00:00.209877
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.179586
  └ 전체 경과   : 0:00:00.389463
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000032
  └ 전체 경과   : 0:00:00.389495
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.466720
  └ 전체 경과   : 0:00:00.856215
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.087467
  └ 전체 경과   : 0:00:00.943682
[08] 파일 -> Markdown
  ├ 전

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 0d78ccb1-5eb4-4045-975b-3303a36117d6
[('청주산남2-1 영구임대주택 예비입주자 모집 공고(2025.11.21)', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019084&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09', datetime.datetime(2025, 12, 15, 12, 32, 21, 222897), datetime.datetime(2025, 12, 15, 12, 32, 21, 222900), 'LH', '임대', '영구임대', '충청북도', '2025.11.21', '2025.12.19', '공고중', 'OPEN')]
[01] 배치 시작 - 청주산남2-1 영구임대주택 예비입주자 모집 공고(2025.11.21)
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.092652
  └ 전체 경과   : 0:00:00.092652
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.133148
  └ 전체 경과   : 0:00:00.225800
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.200100
  └ 전체 경과   : 0:00:00.425900
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000033
  └ 전체 경과   : 0:00:00.425933
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.428811
  └ 전체 경과   : 0:00:00.854744
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.094127
  └ 전체 경과   : 0:00:00.948871
[08] 파일 -> Markdown
  ├ 전 단계 대

...

Started parsing the file under job_id 3ebc091f-91a6-4ea6-9ef4-f30862fe3077
[('2025년 신혼·신생아 전세임대 I 입주자 수시모집 공고', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300017958&ccrCnntSysDsCd=03&uppAisTpCd=13&aisTpCd=17', datetime.datetime(2025, 12, 15, 12, 47, 22, 103703), datetime.datetime(2025, 12, 15, 12, 47, 22, 103709), 'LH', '임대', '전세임대', '서울특별시 외', '2025.10.13', '2025.12.31', '접수중', 'OPEN')]
[01] 배치 시작 - 2025년 신혼·신생아 전세임대 I 입주자 수시모집 공고
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.091750
  └ 전체 경과   : 0:00:00.091750
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.152151
  └ 전체 경과   : 0:00:00.243901
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.180799
  └ 전체 경과   : 0:00:00.424700
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000033
  └ 전체 경과   : 0:00:00.424733
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.316377
  └ 전체 경과   : 0:00:00.741110
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.090412
  └ 전체 경과   : 0:00:00.831522
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:00:16.

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id c9db2285-599b-401d-9dbe-974091bbcdf7


PdfReadError("Invalid Elementary Object starting with b'\\xbf' @661096: b'ps://apply.lh.or.kr)\\xbf\\xa1\\xbc\\xad)\\n/S /URI >>\\n/H /I\\n/F 28\\n>>\\nendobj\\n183 0 obj\\n[\\n182 0 R\\n]'")
TimeoutException('PDF 파싱 타임아웃')


[('군산나운4 영구임대주택 입주자격완화 예비입주자 모집 공고', 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300019167&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=09', datetime.datetime(2025, 12, 15, 12, 47, 41, 160094), datetime.datetime(2025, 12, 15, 12, 47, 41, 160095), 'LH', '임대', '영구임대', '전북특별자치도', '2025.12.12', '2025.12.24', '공고중', 'OPEN')]
[01] 배치 시작 - 군산나운4 영구임대주택 입주자격완화 예비입주자 모집 공고
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.136936
  └ 전체 경과   : 0:00:00.136936
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.157111
  └ 전체 경과   : 0:00:00.294047
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.262191
  └ 전체 경과   : 0:00:00.556238
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000037
  └ 전체 경과   : 0:00:00.556275
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.633801
  └ 전체 경과   : 0:00:01.190076
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.128437
  └ 전체 경과   : 0:00:01.318513
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:03:23.779402
  └ 전체 경과   : 0:03:25.097915
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:0

In [31]:
# for row in annc_list_lh:
#     time_laps = process(row,"LH")

#     print_time_laps(time_laps)
